In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e12/sample_submission.csv
/kaggle/input/playground-series-s5e12/train.csv
/kaggle/input/playground-series-s5e12/test.csv


In [68]:
train_diabetes=pd.read_csv("/kaggle/input/playground-series-s5e12/train.csv")
train_diabetes[23:200][:]

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
23,23,53,2,29,8.1,6.4,6.3,27.3,0.83,128,...,Female,White,Highschool,Low,Never,Employed,0,0,0,1.0
24,24,59,2,72,4.5,8.7,3.7,22.9,0.84,116,...,Female,White,No formal,Middle,Current,Retired,0,0,0,1.0
25,25,52,2,46,5.3,8.0,6.0,24.0,0.88,131,...,Male,White,Graduate,Middle,Never,Employed,1,0,0,1.0
26,26,73,1,52,8.3,7.5,7.9,24.9,0.86,137,...,Female,Hispanic,Highschool,Upper-Middle,Former,Employed,1,0,0,0.0
27,27,61,1,118,6.3,8.5,7.3,25.0,0.90,101,...,Female,White,Highschool,Low,Never,Employed,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,47,1,46,5.0,8.2,6.3,25.3,0.88,103,...,Male,White,Highschool,Lower-Middle,Former,Retired,0,0,0,1.0
196,196,41,4,81,3.6,5.9,7.2,29.4,0.89,117,...,Female,Black,Graduate,Middle,Former,Employed,0,1,0,0.0
197,197,37,2,98,4.8,6.8,7.3,24.9,0.88,115,...,Female,White,Graduate,Upper-Middle,Former,Employed,0,0,0,1.0
198,198,79,2,88,3.9,7.7,4.0,32.6,0.87,139,...,Female,White,Highschool,Lower-Middle,Never,Employed,0,1,0,1.0


In [69]:
y=train_diabetes.loc[:,"diagnosed_diabetes"]
y.head()

0    1.0
1    1.0
2    0.0
3    1.0
4    1.0
Name: diagnosed_diabetes, dtype: float64

In [70]:
train_diabetes.drop("id",axis=1,inplace=True)
train_diabetes.head()

,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,...,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,70,...,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,77,...,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,89,...,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0
3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,69,...,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0,1.0
4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,60,...,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0,1.0


In [79]:
#filling all the NA values in the df
def fill_null(train_diabetes):
    train_diabetes['age']=train_diabetes['age'].fillna(train_diabetes['age'].mean())
    train_diabetes["alcohol_consumption_per_week"]=train_diabetes["alcohol_consumption_per_week"].fillna(train_diabetes["alcohol_consumption_per_week"].median())
    train_diabetes["physical_activity_minutes_per_week"]=train_diabetes["physical_activity_minutes_per_week"].fillna(train_diabetes["physical_activity_minutes_per_week"].mean())
    train_diabetes['diet_score']=train_diabetes["diet_score"].fillna(train_diabetes["diet_score"].mean())
    train_diabetes["sleep_hours_per_day"]=train_diabetes["sleep_hours_per_day"].fillna(train_diabetes["sleep_hours_per_day"].mean())
    train_diabetes['screen_time_hours_per_day']=train_diabetes["screen_time_hours_per_day"].fillna(train_diabetes["screen_time_hours_per_day"].mean())
    train_diabetes["bmi"]=train_diabetes["bmi"].fillna(train_diabetes["bmi"].mean())
    train_diabetes["waist_to_hip_ratio"]=train_diabetes["waist_to_hip_ratio"].fillna(train_diabetes["waist_to_hip_ratio"].mean())
    train_diabetes["systolic_bp"]=train_diabetes["systolic_bp"].fillna(120)
    train_diabetes["diastolic_bp"]=train_diabetes["diastolic_bp"].fillna(80)
    train_diabetes["triglycerides"]=train_diabetes["triglycerides"].fillna(150)
    train_diabetes["gender"]=train_diabetes["gender"].fillna("Other")
    train_diabetes["ethnicity"]=train_diabetes["ethnicity"].fillna("None_given")
    train_diabetes["education_level"]=train_diabetes["education_level"].fillna("Highschool")
    train_diabetes["income_level"]=train_diabetes["income_level"].fillna("Lower-Middle")
    train_diabetes["smoking_status"]=train_diabetes["smoking_status"].fillna("Never")
    train_diabetes["employment_status"]=train_diabetes["employment_status"].fillna("Employed")
    train_diabetes["family_history_diabetes"]=train_diabetes["family_history_diabetes"].fillna(train_diabetes["family_history_diabetes"].mode()[0])
    train_diabetes["hypertension_history"]=train_diabetes["hypertension_history"].fillna(train_diabetes["hypertension_history"].mode()[0])
    train_diabetes["cardiovascular_history"]=train_diabetes["cardiovascular_history"].fillna(train_diabetes["cardiovascular_history"].mode()[0])
    columns=["gender", "ethnicity", "education_level", 
                   "income_level", "smoking_status", "employment_status"]
    for col in columns:
        train_diabetes[col] = train_diabetes[col].astype("category")
    return train_diabetes

In [72]:
#now extract all the dtypes with all the strings as their values and then assign them categories through pandas 
##for the xgbboost which only accepts the numerical data historically we sends them categories which again btw we need bins or histograms during the model prediction
columns=["gender", "ethnicity", "education_level", 
                   "income_level", "smoking_status", "employment_status"]
train_diabetes=fill_null(train_diabetes)
for col in columns:
    train_diabetes[col] = train_diabetes[col].astype("category")

x=train_diabetes.drop("diagnosed_diabetes",axis=1)
x.head()

,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,...,triglycerides,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history
0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,70,...,102,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0
1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,77,...,124,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0
2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,89,...,108,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0
3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,69,...,123,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0
4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,60,...,124,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0


In [73]:
model=xgb.XGBClassifier(tree_method="hist",enable_categorical=True)
model.fit(x,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [84]:
test=pd.read_csv("/kaggle/input/playground-series-s5e12/test.csv")
test_ids=test["id"]
X=test.drop("id",axis=1)
X=fill_null(X)
preds=model.predict_proba(X)[:,1]
submission=pd.DataFrame({
    "id":test_ids,
    "diagnosed_diabetes":preds
})
submission.to_csv("submission.csv",index=False)